In [571]:
import json
import numpy as np

In [572]:
def linear(x):
    return x

def derivative_linear(x):
    return np.ones(x.shape)

def relu(x):
    return np.maximum(0, x)

def derivative_relu(x):
    return np.where(x < 0, 0, 1)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def derivative_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

def softmax(x):
    x = np.array(x)
    if len(x.shape) == 1:
        x = x.reshape(1, -1)
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def derivative_softmax(x, isTarget):
    if isTarget:
        return softmax(x)
    else:
        return -(1-softmax(x))

In [573]:
def delta_output(activation_function, target, output, net):
  if activation_function == "linear":
      result = (target - output) * derivative_linear(net)
  elif activation_function == "relu":
      result = (target - output) * derivative_relu(net)
  elif activation_function == "sigmoid":
      result = (target - output) * derivative_sigmoid(net)
  elif activation_function == "softmax":
      result = -derivative_softmax(net, True)

  return result

def delta_hidden(activation_function, net, delta_output, weights):
  if activation_function == "linear":
        delta_output_np = np.array(delta_output)
        weights_np = np.array(weights.T)
        result = np.sum(-np.dot(delta_output_np, weights_np) * derivative_linear(net))
  elif activation_function == "relu":
      delta_output_np = np.array(delta_output)
      weights_np = np.array(weights.T)
      result = np.sum(-np.dot(delta_output_np, weights_np) * derivative_relu(net))
  elif activation_function == "sigmoid":
      result = np.dot(delta_output, weights.T) * derivative_sigmoid(net)
  elif activation_function == "softmax":
      result = -derivative_softmax(net, False)

  return result

def calculate_delta(activation_function, learning_rate, x, target, output, net, weights, isOutputLayer, out = 1):
    if isOutputLayer:
        return learning_rate * delta_output(activation_function, target, output, net) * x
    else:
        return learning_rate * delta_hidden(activation_function, net, out, weights) * x

In [574]:
def loss_function_sse(target, output):
    e = 0.5 * np.sum((target - output) ** 2)
    return e

def loss_function_softmax(target, output):
    e = -np.sum(target * np.log(output))
    return e

In [575]:
def terminate_condition(stopped_by, max_iteration, error_threshold, iteration, error):
    if stopped_by == "max_iteration":
        return iteration >= max_iteration
    elif stopped_by == "error_threshold":
        return error <= error_threshold

def forward_propagation(model, input_data, weights):
    layers = model["layers"]
    output_layer = [input_data]

    for i, layer in enumerate(layers):
        activation_function = layer["activation_function"]
        weight_matrix = weights[i]
        bias = layer.get("bias", 1)

        input_activation = output_layer[-1]
        # print("Dimensi input_activation:", input_activation.shape)
        # print(input_activation)
        if len(input_activation.shape) == 1:
            input_activation_with_bias = np.insert(input_activation, 0, bias)
            input_activation_with_bias = input_activation_with_bias.reshape(1, -1)
        else:
            batch_size = input_activation.shape[0]
            bias_vector = np.ones((batch_size, 1)) * bias
            input_activation_with_bias = np.concatenate((bias_vector, input_activation), axis=1)
        # print("Dimensi input_activation_with_bias:", input_activation_with_bias.shape)
        # print(input_activation_with_bias)
        # print("Dimensi weight_matrix:", weight_matrix.shape)
        # print(weight_matrix)

        output_linear_combination = np.dot(input_activation_with_bias, weight_matrix)

        if activation_function == "linear":
            activation_result = linear(output_linear_combination)
        elif activation_function == "relu":
            activation_result = relu(output_linear_combination)
        elif activation_function == "sigmoid":
            activation_result = sigmoid(output_linear_combination)
        elif activation_function == "softmax":
            activation_result = softmax(output_linear_combination)

        output_layer.append(activation_result)

    return output_layer

def back_propagation(model, input, initial_weights, target_input, learning_rate, batch_size, stopped_by, max_iteration, error_threshold):

    epoch=0
    error = np.inf

    while not terminate_condition(stopped_by, max_iteration, error_threshold, epoch, error) :
        epoch += 1

        if error == np.inf:
            error = 0

        for batch in range(int(input.shape[0]/batch_size)):
            input_batch = input[batch*batch_size:(batch+1)*batch_size]
            target_batch = target_input[batch*batch_size:(batch+1)*batch_size]
            output_forward = forward_propagation(model, input_batch, initial_weights)
            
            delta_weight = [np.zeros(w.shape) for w in initial_weights]
            for id_data in range(batch_size):
                # Calculate error each output unit
                id = id_data + batch_size * batch
                
                '''
                if (model["layers"][-1]["activation_function"] == "softmax"):
                    all_net = []
                    # Iterate through each output unit
                    for i in range(len(output_forward[-1][id])):
                        input_x = np.insert(output_forward[-2], 0, 1)
                        all_net.append(np.dot(input_x, initial_weights[-1][:,i]))
                '''
                error_k = []
                error_k_divided = []
                # Iterate through each output unit
                for i in range(len(output_forward[-1][id_data])):
                    activation_function = model["layers"][-1]["activation_function"]
                    output = output_forward[-1][id_data][i]
                    target = target_batch[id_data][i]
                    
                    input_x = output_forward[-2][id_data]
                    input_x = np.insert(input_x, 0, 1)
                    
                    net = np.dot(input_x.T, initial_weights[-1][:,i])

                    weights = initial_weights[-1]
                    error_k_j = []
                    error_k_j_divided = []
                    for j in range(input_x.shape[0]):
                        x = input_x[j]
                        
                        if (activation_function == "softmax"):
                            error_k_j_one_unit = calculate_delta(activation_function, learning_rate, x, target, output, net, weights, True)
                            error_k_j_one_unit_divided = error_k_j_one_unit/(x * learning_rate)
                            error_k_j_divided = (error_k_j_one_unit_divided)
                            error_k_j.append(error_k_j_one_unit)
                        else:
                            error_k_j_one_unit = calculate_delta(activation_function, learning_rate, x, target, output, net, weights, True)
                            error_k_j_one_unit_divided = error_k_j_one_unit/(x * learning_rate)
                            error_k_j_divided = (error_k_j_one_unit_divided)
                            error_k_j.append(error_k_j_one_unit)

                    error_k.append(error_k_j)
                    error_k_divided.append(error_k_j_divided)
                        
                # Calculate error each hidden unit
                error_h = []
                
                # Iterate through each hidden layer
                for j in range(len(output_forward)-2, 0, -1):
                    error_h_j = []
                    error_h_j_divided = [] 
                    activation_function = model["layers"][j - 1]["activation_function"]

                    # Iterate through each hidden layer unit
                    for k in range(len(output_forward[j][id_data])):
                        output = output_forward[j][id_data][k]
                        target = target_batch[id_data][k]
                        
                        input_x = output_forward[j-1][id_data]
                        input_x = np.insert(input_x, 0, 1)
                        
                        net = np.dot(input_x.T, initial_weights[j-1][:,k])

                        weights = initial_weights[j-1]

                        error_h_j_k = []
                        error_h_j_k_divided = []
                        for l in range(input_x.shape[0]):
                            x = input_x[l]
                            
                            if (activation_function == "softmax"):
                                error_h_j_k_one_unit = calculate_delta(activation_function, learning_rate, x, target, output, net, weights, False)
                                error_h_j_k_one_unit_divided = error_h_j_k_one_unit/(x * learning_rate)
                                error_h_j_k.append(error_h_j_k_one_unit)
                            else:
                                if (len(error_h) == 0):
                                    error_h_j_k_one_unit = calculate_delta(activation_function, learning_rate, x, target, output, net, weights, False, error_k_divided)
                                    error_h_j_k_one_unit_divided = error_h_j_k_one_unit/(x * learning_rate)
                                    error_h_j_k_divided = (error_h_j_k_one_unit_divided)
                                    error_h_j_k.append(error_h_j_k_one_unit)
                                else:
                                    error_h_j_k_one_unit = calculate_delta(activation_function, learning_rate, x, target, output, net, weights, False, error_h_j_divided[-1])
                                    error_h_j_k_one_unit_divided = error_h_j_k_one_unit/(x * learning_rate)
                                    error_h_j_k_divided = (error_h_j_k_one_unit_divided)
                                    error_h_j_k.append(error_h_j_k_one_unit)

                        error_h_j.append(error_h_j_k)
                        error_h_j_divided.append(error_h_j_k_divided)
                    error_h.append(error_h_j)

                for j in range(len(initial_weights)-1, -1, -1):
                    if j == len(initial_weights)-1:
                        for k in range(len(initial_weights[j])):
                            for l in range(len(initial_weights[j][k])):
                                # print("delta_weight[j][k][l]", delta_weight[j][k][l])
                                # print("error_k[l][k]", error_k[l][k])
                                # print("sum", delta_weight[j][k][l] + error_k[l][k])
                                delta_weight[j][k][l] += error_k[l][k]
                    else:
                        for k in range(len(initial_weights[j])):
                            for l in range(len(initial_weights[j][k])):
                                delta_weight[j][k][l] += error_h[j][l][k]
            # Update weight per batch
            # sum delta weight
            for i in range(len(initial_weights)):
                for j in range(len(initial_weights[i])):
                    for k in range(len(initial_weights[i][j])):
                        initial_weights[i][j][k] += delta_weight[i][j][k]
            
            # print("delta_weight",delta_weight)

        # Calculate error
        for data in range(input.shape[0]):
            output_forward = forward_propagation(model, input, initial_weights)

            activation_function = model["layers"][-1]["activation_function"]
            if activation_function == "softmax":
                error += loss_function_softmax(target_input[data], output_forward[-1][data])
            else:
                error += loss_function_sse(target_input[data], output_forward[-1][data])

    return initial_weights

In [576]:
# Load JSON input
with open("test/backpropagation/mlp.json", "r") as file:
    data = json.load(file)

# Extract data
model = data["case"]["model"]
input_data = np.array(data["case"]["input"])
initial_weights = [np.array(layer_weights) for layer_weights in data["case"]["initial_weights"]]
target = np.array(data["case"]["target"])
learning_rate = data["case"]["learning_parameters"]["learning_rate"]
batch_size = data["case"]["learning_parameters"]["batch_size"]
max_iteration = data["case"]["learning_parameters"]["max_iteration"]
error_threshold = data["case"]["learning_parameters"]["error_threshold"]
stopped_by = data["expect"]["stopped_by"]
final_weights = [np.array(layer_weights) for layer_weights in data["expect"]["final_weights"]]

result = back_propagation(model, input_data, initial_weights, target, learning_rate, batch_size, stopped_by, max_iteration, error_threshold)
print(result)

[array([[-0.02056 ,  0.07944 ],
       [-0.30616 ,  0.49384 ],
       [ 0.502608,  0.602608]]), array([[ 0.2748  ,  0.188   ],
       [ 0.435904, -0.53168 ],
       [ 0.68504 ,  0.7824  ]])]


In [577]:
output_layer = forward_propagation(model, input_data, initial_weights)
print(output_layer)

[array([[-1. ,  0.2],
       [ 0.2, -1. ]]), array([[ 0.3861216, -0.2938784],
       [-0.5844   , -0.4244   ]]), array([[0.24179349, 0.        ],
       [0.        , 0.16666323]])]


In [578]:
# backward propagation with sklearn
from sklearn.neural_network import MLPClassifier

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

# Load dataset
dataset = pd.read_csv("test/backpropagation/iris.csv")
learning_data = dataset.drop(columns=["Species"])
target_data = dataset["Species"]

mlp = MLPClassifier(hidden_layer_sizes=(5,9,3), max_iter=1000, activation='logistic', learning_rate='constant', learning_rate_init=0.01,)
mlp.fit(learning_data, target_data)

y_pred = mlp.predict(learning_data)
print("Predicted values:")
print(y_pred)
print("Actual values:")
print(target_data)
print("Accuracy:", mlp.score(learning_data, target_data))

Predicted values:
['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor